In [1]:
#Charger les fichiers BDPM
import pandas as pd

cis = pd.read_csv("data/CIS_bdpm.txt", 
                  sep="\t", header=None, dtype=str, encoding="latin-1")

compo = pd.read_csv("data/CIS_COMPO_bdpm.txt",
                    sep="\t", header=None, dtype=str, encoding="latin-1")

mitm = pd.read_csv("data/CIS_MITM.txt",
                   sep="\t", header=None, dtype=str, encoding="latin-1")


### Donner des noms aux colonnes

In [2]:

cis.columns = [
    "cis", "nom_commercial", "forme", "voie",
    "statut_amm", "procedure_amm", "etat_commercialisation",
    "date_amm", "statut_bdm", "num_amm_eu",
    "titulaire", "surveillance"
]


In [3]:
cis

,cis,nom_commercial,forme,voie,statut_amm,procedure_amm,etat_commercialisation,date_amm,statut_bdm,num_amm_eu,titulaire,surveillance
0,61266250,"A 313 200 000 UI POUR CENT, pommade",pommade,cutanée,Autorisation active,Procédure nationale,Commercialisée,12/03/1998,NaN,NaN,PHARMA DEVELOPPEMENT,Non
1,62869109,"A 313 50 000 U.I., capsule molle",capsule molle,orale,Autorisation active,Procédure nationale,Commercialisée,07/07/1997,NaN,NaN,PHARMA DEVELOPPEMENT,Non
2,69103878,"A.D.N. BOIRON, degré de dilution compris entre...",comprimé et solution(s) et granules et poudre...,cutanée;orale;sublinguale,Autorisation active,Enreg homéo (Proc. Nat.),Commercialisée,14/10/2021,NaN,NaN,BOIRON,Non
3,61876780,"ABACAVIR ARROW 300 mg, comprimé pelliculé sécable",comprimé pelliculé sécable,orale,Autorisation active,Procédure décentralisée,Commercialisée,22/10/2019,NaN,NaN,ARROW GENERIQUES,Non
4,63797011,"ABACAVIR SANDOZ 300 mg, comprimé pelliculé séc...",comprimé pelliculé sécable,orale,Autorisation active,Procédure décentralisée,Commercialisée,30/12/2016,NaN,NaN,SANDOZ,Non
...,...,...,...,...,...,...,...,...,...,...,...,...
15816,64949486,"ZYRTECSET 10 mg, comprimé pelliculé sécable",comprimé pelliculé sécable,orale,Autorisation active,Procédure nationale,Commercialisée,28/10/1994,NaN,NaN,UCB PHARMA,Non
15817,67337081,"ZYTIGA 500 mg, comprimé pelliculé",comprimé pelliculé,orale,Autorisation active,Procédure centralisée,Commercialisée,09/11/2016,NaN,EU/1/11/714,JANSSEN CILAG INTERNATIONAL NV,Non
15818,63095061,"ZYVOXID 100 mg/5 ml, granulés pour suspension ...",granulés pour suspension buvable,orale,Autorisation active,Procédure de reconnaissance mutuelle,Commercialisée,31/08/2001,NaN,NaN,PFIZER HOLDING FRANCE,Non
15819,63283095,"ZYVOXID 2 mg/ml, solution pour perfusion",solution pour perfusion,intraveineuse,Autorisation active,Procédure de reconnaissance mutuelle,Commercialisée,31/08/2001,NaN,NaN,PFIZER HOLDING FRANCE,Non


In [4]:
compo.columns = [
    "cis", "element", "code_substance", "dci",
    "dosage", "reference_dosage", "nature", "num_liaison"
]


In [5]:
compo

,cis,element,code_substance,dci,dosage,reference_dosage,nature,num_liaison
0,60002283,comprimé,42215,ANASTROZOLE,"1,00 mg",un comprimé,SA,1
1,60002746,granules,05319,ACTAEA RACEMOSA POUR PRÉPARATIONS HOMÉOPATHIQUES,2CH à 30CH et 4DH à 60DH,un comprimé,SA,7
2,60002746,solution buvable en gouttes,05319,ACTAEA RACEMOSA POUR PRÉPARATIONS HOMÉOPATHIQUES,2CH à 30CH et 4DH à 60DH,un comprimé,SA,8
3,60003620,suspension,04179,DIPROPIONATE DE BÉCLOMÉTASONE,800 microgrammes,2 ml de suspension,SA,1
4,60004277,gélule,03902,FÉNOFIBRATE,"100,00 mg",une gélule,SA,1
...,...,...,...,...,...,...,...,...
32558,69998996,solution,03192,GLUCONATE DE CUIVRE,1428 microgrammes,une ampoule de 10 ml,SA,2
32559,69998996,solution,03904,GLUCONATE DE MANGANÈSE,"40,52 microgrammes",une ampoule de 10 ml,SA,3
32560,69998996,solution,05246,GLUCONATE DE ZINC,6970 microgrammes,une ampoule de 10 ml,SA,1
32561,69998996,solution,30108,SÉLÉNITE DE SODIUM,"43,81 microgrammes",une ampoule de 10 ml,SA,6


In [6]:
mitm.columns = [
    "cis", "atc", "denomination", "lien_bdpm"
]


In [7]:
mitm

,cis,atc,denomination,lien_bdpm
0,68053454,A02BA01,"CIMETIDINE ARROW 200 mg, comprimé effervescent",https://base-donnees-publique.medicaments.gouv...
1,60005856,A02BA03,"FAMOTIDINE EG 20 mg, comprimé pelliculé",https://base-donnees-publique.medicaments.gouv...
2,67557907,A02BA03,"FAMOTIDINE EG 40 mg, comprimé pelliculé",https://base-donnees-publique.medicaments.gouv...
3,69606819,A02BC01,"MOPRAL 10 mg, gélule gastro-résistante",https://base-donnees-publique.medicaments.gouv...
4,64103828,A02BC01,"MOPRAL 20 mg, gélule gastro-résistante",https://base-donnees-publique.medicaments.gouv...
...,...,...,...,...
8102,60447461,V09XA01,NORCHOLESTEROL IODE [131I] CIS BIO INTERNATION...,https://base-donnees-publique.medicaments.gouv...
8103,67041506,V10XA01,IODURE (131 I) DE SODIUM POUR THERAPIE CURIUMP...,https://base-donnees-publique.medicaments.gouv...
8104,69279253,V10XA01,"THERACAP131, Iodure(131I) de sodium pour théra...",https://base-donnees-publique.medicaments.gouv...
8105,66302387,V10XX03,"XOFIGO 1100 kBq/mL, solution injectable",https://base-donnees-publique.medicaments.gouv...


### Garder seulement les substances actives (DCI)

In [8]:
compo_sa = compo[compo["nature"] == "SA"]


### Nom commercial + DCI + ATC

In [9]:
df = cis.merge(compo_sa[["cis", "dci"]], on="cis", how="left")
df = df.merge(mitm[["cis", "atc"]], on="cis", how="left")


In [10]:
df

,cis,nom_commercial,forme,voie,statut_amm,procedure_amm,etat_commercialisation,date_amm,statut_bdm,num_amm_eu,titulaire,surveillance,dci,atc
0,61266250,"A 313 200 000 UI POUR CENT, pommade",pommade,cutanée,Autorisation active,Procédure nationale,Commercialisée,12/03/1998,NaN,NaN,PHARMA DEVELOPPEMENT,Non,"CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",NaN
1,62869109,"A 313 50 000 U.I., capsule molle",capsule molle,orale,Autorisation active,Procédure nationale,Commercialisée,07/07/1997,NaN,NaN,PHARMA DEVELOPPEMENT,Non,"CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",NaN
2,69103878,"A.D.N. BOIRON, degré de dilution compris entre...",comprimé et solution(s) et granules et poudre...,cutanée;orale;sublinguale,Autorisation active,Enreg homéo (Proc. Nat.),Commercialisée,14/10/2021,NaN,NaN,BOIRON,Non,A.D.N. POUR PRÉPARATIONS HOMÉOPATHIQUES,NaN
3,69103878,"A.D.N. BOIRON, degré de dilution compris entre...",comprimé et solution(s) et granules et poudre...,cutanée;orale;sublinguale,Autorisation active,Enreg homéo (Proc. Nat.),Commercialisée,14/10/2021,NaN,NaN,BOIRON,Non,A.D.N. POUR PRÉPARATIONS HOMÉOPATHIQUES,NaN
4,69103878,"A.D.N. BOIRON, degré de dilution compris entre...",comprimé et solution(s) et granules et poudre...,cutanée;orale;sublinguale,Autorisation active,Enreg homéo (Proc. Nat.),Commercialisée,14/10/2021,NaN,NaN,BOIRON,Non,A.D.N. POUR PRÉPARATIONS HOMÉOPATHIQUES,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27027,64949486,"ZYRTECSET 10 mg, comprimé pelliculé sécable",comprimé pelliculé sécable,orale,Autorisation active,Procédure nationale,Commercialisée,28/10/1994,NaN,NaN,UCB PHARMA,Non,CÉTIRIZINE (DICHLORHYDRATE DE),NaN
27028,67337081,"ZYTIGA 500 mg, comprimé pelliculé",comprimé pelliculé,orale,Autorisation active,Procédure centralisée,Commercialisée,09/11/2016,NaN,EU/1/11/714,JANSSEN CILAG INTERNATIONAL NV,Non,ACÉTATE D'ABIRATÉRONE,L02BX03
27029,63095061,"ZYVOXID 100 mg/5 ml, granulés pour suspension ...",granulés pour suspension buvable,orale,Autorisation active,Procédure de reconnaissance mutuelle,Commercialisée,31/08/2001,NaN,NaN,PFIZER HOLDING FRANCE,Non,LINÉZOLIDE,J01XX08
27030,63283095,"ZYVOXID 2 mg/ml, solution pour perfusion",solution pour perfusion,intraveineuse,Autorisation active,Procédure de reconnaissance mutuelle,Commercialisée,31/08/2001,NaN,NaN,PFIZER HOLDING FRANCE,Non,LINÉZOLIDE,J01XX08


### Regrouper les DCI par médicament

In [11]:
df_grouped = df.groupby(["cis", "nom_commercial", "atc"])["dci"].apply(
    lambda x: " + ".join(sorted(set(x)))
).reset_index()


In [12]:
df_grouped

,cis,nom_commercial,atc,dci
0,60003620,BECLOSPIN 800 microgrammes/2ml suspension pour...,R03BA01,DIPROPIONATE DE BÉCLOMÉTASONE
1,60004277,"FENOFIBRATE TEVA 100 mg, gélule",C10AB05,FÉNOFIBRATE
2,60004487,"TRAMADOL EG L.P. 200 mg, comprimé à libération...",N02AX02,CHLORHYDRATE DE TRAMADOL
3,60005856,"FAMOTIDINE EG 20 mg, comprimé pelliculé",A02BA03,FAMOTIDINE
4,60007437,"ATORVASTATINE TEVA SANTE 40 mg, comprimé pelli...",C10AA05,ATORVASTATINE CALCIQUE
...,...,...,...,...
8101,69994784,"MYALEPTA 11,3 mg, poudre pour solution injectable",A16AA07,MÉTRÉLEPTINE
8102,69995434,"TRAMADOL/PARACETAMOL ARROW 37,5 mg/325 mg, com...",N02AJ13,CHLORHYDRATE DE TRAMADOL + PARACÉTAMOL
8103,69998003,"DIAZEPAM ACCORD 10 mg, comprimé",N05BA01,DIAZÉPAM
8104,69998156,"SPORANOX 10 mg/mL, solution buvable",J02AC02,ITRACONAZOLE


In [13]:
vih = df_grouped

#### Filtrer uniquement les médicaments VIH

In [21]:
vih = df_grouped[df_grouped["atc"].str.startswith("J05", na=False)]


In [14]:
vih

,cis,nom_commercial,atc,dci
0,60003620,BECLOSPIN 800 microgrammes/2ml suspension pour...,R03BA01,DIPROPIONATE DE BÉCLOMÉTASONE
1,60004277,"FENOFIBRATE TEVA 100 mg, gélule",C10AB05,FÉNOFIBRATE
2,60004487,"TRAMADOL EG L.P. 200 mg, comprimé à libération...",N02AX02,CHLORHYDRATE DE TRAMADOL
3,60005856,"FAMOTIDINE EG 20 mg, comprimé pelliculé",A02BA03,FAMOTIDINE
4,60007437,"ATORVASTATINE TEVA SANTE 40 mg, comprimé pelli...",C10AA05,ATORVASTATINE CALCIQUE
...,...,...,...,...
8101,69994784,"MYALEPTA 11,3 mg, poudre pour solution injectable",A16AA07,MÉTRÉLEPTINE
8102,69995434,"TRAMADOL/PARACETAMOL ARROW 37,5 mg/325 mg, com...",N02AJ13,CHLORHYDRATE DE TRAMADOL + PARACÉTAMOL
8103,69998003,"DIAZEPAM ACCORD 10 mg, comprimé",N05BA01,DIAZÉPAM
8104,69998156,"SPORANOX 10 mg/mL, solution buvable",J02AC02,ITRACONAZOLE


### Nettoyer le DataFrame final

In [15]:
vih = vih[["nom_commercial", "dci", "atc"]].drop_duplicates()
vih = vih.sort_values("nom_commercial")


In [27]:
dictionnaire_vih = vih.to_dict(orient="records")


In [21]:
vih.to_excel("data/vih.xlsx", index=False)


In [20]:
mot = "NORVIR"

mask = vih.astype(str).apply(lambda col: col.str.contains(mot, case=False, na=False))
resultat = vih[mask.any(axis=1)]

resultat

,nom_commercial,dci,atc
6400,"NORVIR 100 mg, comprimé pelliculé",RITONAVIR,J05AE03
4186,"NORVIR 100 mg, poudre pour suspension buvable",RITONAVIR,J05AE03
